<a href="https://colab.research.google.com/github/vifirsanova/compling/blob/main/llms/langchain_youtube_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Загрузка библиотек

In [ ]:
! pip install -q -U langchain langchain-community youtube_transcript_api bitsandbytes accelerate
! pip install --upgrade tensorflow

In [2]:
import torch
from transformers import pipeline
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_community.document_loaders import YoutubeLoader

# Загрузка видео

In [3]:
loader = YoutubeLoader.from_youtube_url("https://youtu.be/-WBPm68eB9E", language='ru')
transcript = loader.load()
transcript

[Document(page_content='соцсети небезопасны Травля угрозы преследования по данным социологических исследований более 10% пользователей интернета сталкивались с ситуациями представляющими реальную опасность и не менее 40% хотя бы раз страдали от нарушения личных границ в онлайн пространстве соцсети недоступны для всех по данным всемирной организации здравоохранения каждый шестой человек в мире имеет ограничение здоровья чтобы интернет доступным для каждого консорциум всемирной паутины разработал минимальные требования руководство по обеспечению доступности веб-контенту 97% веб-страниц игнорируют эти требования такими сайтами не может пользоваться человек с ограничениями слуха зрения опорнодвигательного аппарата ментальными особенностями социальные сети вызывают зависимость научно доказано что яркие привлекательные интерфейсы мобильных приложений прово в нашем мозге выработку гормона дофамина гормона удовольствия результат настоящая зависимость наш организм постоянно просит листать ленту

# Загрузка модели

In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [36]:
model_4bit = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    device_map="auto",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True
        ))

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

pipe = pipeline("text-generation", model=model_4bit, tokenizer=tokenizer, max_new_tokens=100)
hf = HuggingFacePipeline(pipeline=pipe)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=50)
texts = text_splitter.split_documents(transcript)

In [17]:
texts

[Document(page_content='соцсети небезопасны Травля угрозы преследования по данным социологических исследований более 10% пользователей интернета сталкивались с ситуациями представляющими реальную опасность и не менее 40% хотя бы раз страдали от нарушения личных границ в онлайн пространстве соцсети недоступны для всех по данным всемирной организации здравоохранения каждый шестой человек в мире имеет ограничение здоровья чтобы интернет доступным для каждого консорциум всемирной паутины разработал минимальные требования руководство по обеспечению доступности веб-контенту 97% веб-страниц игнорируют эти требования такими сайтами не может пользоваться человек с ограничениями слуха зрения опорнодвигательного аппарата ментальными особенностями социальные сети вызывают зависимость научно доказано что яркие привлекательные интерфейсы мобильных приложений прово в нашем мозге выработку гормона дофамина гормона удовольствия результат настоящая зависимость наш организм постоянно просит листать ленту

In [52]:
from langchain.chains.summarize import load_summarize_chain

chain = load_summarize_chain(llm=hf, chain_type='stuff', verbose=True)
print(chain.llm_chain.prompt.template)

Write a concise summary of the following:


"{text}"


CONCISE SUMMARY:


In [39]:
summary = chain.run(input_documents=texts)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"соцсети небезопасны Травля угрозы преследования по данным социологических исследований более 10% пользователей интернета сталкивались с ситуациями представляющими реальную опасность и не менее 40% хотя бы раз страдали от нарушения личных границ в онлайн пространстве соцсети недоступны для всех по данным всемирной организации здравоохранения каждый шестой человек в мире имеет ограничение здоровья чтобы интернет доступным для каждого консорциум всемирной паутины разработал минимальные требования руководство по обеспечению доступности веб-контенту 97% веб-страниц игнорируют эти требования такими сайтами не может пользоваться человек с ограничениями слуха зрения опорнодвигательного аппарата ментальными особенностями социальные сети вызывают зависимость научно доказано что яркие привлекательные интерфейсы мобильных приложений прово в нашем мо

In [42]:
summary[-500:]

':\n\nThe text discusses the dangers of social media and the addiction it can cause. It also talks about the importance of accessibility in web content and the need for guidelines to ensure it. The text describes a project called "МПИ" (MPI) which is a closed system that requires an invitation to join. The project aims to create a safe and accessible virtual space for inclusive education. The project uses artificial intelligence to create a friendly virtual assistant named MK to help users navigate'

In [43]:
chain = load_summarize_chain(llm=hf, chain_type='map_reduce', verbose=True)
summary = chain.run(input_documents=texts)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"соцсети небезопасны Травля угрозы преследования по данным социологических исследований более 10% пользователей интернета сталкивались с ситуациями представляющими реальную опасность и не менее 40% хотя бы раз страдали от нарушения личных границ в онлайн пространстве соцсети недоступны для всех по данным всемирной организации здравоохранения каждый шестой человек в мире имеет ограничение здоровья чтобы интернет доступным для каждого консорциум всемирной паутины разработал минимальные требования руководство по обеспечению доступности веб-контенту 97% веб-страниц игнорируют эти требования такими сайтами не может пользоваться человек с ограничениями слуха зрения опорнодвигательного аппарата ментальными особенностями социальные сети вызывают зависимость научно доказано что яркие привлекательные интерфейсы мобильных приложений прово в наше

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"Write a concise summary of the following:


"соцсети небезопасны Травля угрозы преследования по данным социологических исследований более 10% пользователей интернета сталкивались с ситуациями представляющими реальную опасность и не менее 40% хотя бы раз страдали от нарушения личных границ в онлайн пространстве соцсети недоступны для всех по данным всемирной организации здравоохранения каждый шестой человек в мире имеет ограничение здоровья чтобы интернет доступным для каждого консорциум всемирной паутины разработал минимальные требования руководство по обеспечению доступности веб-контенту 97% веб-страниц игнорируют эти требования такими сайтами не может пользоваться человек с ограничениями слуха зрения опорнодвигательного аппарата ментальными особенностями социальные сети вызывают зависимость научно доказано что яркие 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



> Finished chain.

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"Write a concise summary of the following:


"Write a concise summary of the following:


"Write a concise summary of the following:


"Write a concise summary of the following:


"соцсети небезопасны Травля угрозы преследования по данным социологических исследований более 10% пользователей интернета сталкивались с ситуациями представляющими реальную опасность и не менее 40% хотя бы раз страдали от нарушения личных границ в онлайн пространстве соцсети недоступны для всех по данным всемирной организации здравоохранения каждый шестой человек в мире имеет ограничение здоровья чтобы интернет доступным для каждого консорциум всемирной паутины разработал минимальные требования руководство по обеспечению доступности веб-контенту 97% веб-страниц игнорируют эти требования такими сайтами не может пользоваться ч

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



> Finished chain.

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"Write a concise summary of the following:


"Write a concise summary of the following:


"Write a concise summary of the following:


"Write a concise summary of the following:


"вам о миссии нашего проекта и поделюсь частичкой этого удивительного путешествия мой интерес к инклюзии и созданию безопасной доступной среды связан с личные истории я училась в инклюзивной среде и собственными глазами видела С какими трудностями сталкиваются студенты с особыми потребностями эмпатия вот причина по которой появился наш проект миссия эмпи осознанно использовать технологии искусственного интеллекта чтобы создать безопасное виртуальное пространство для поддержки инклюзивного образования стирать границы укреплять развитие общества в котором У каждого есть возможность быть услышанным где каждый включён каждый оце

ValueError: A single document was longer than the context length, we cannot handle this.

In [44]:
chain = load_summarize_chain(llm=hf, chain_type='refine', verbose=True)
summary = chain.run(input_documents=texts)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"соцсети небезопасны Травля угрозы преследования по данным социологических исследований более 10% пользователей интернета сталкивались с ситуациями представляющими реальную опасность и не менее 40% хотя бы раз страдали от нарушения личных границ в онлайн пространстве соцсети недоступны для всех по данным всемирной организации здравоохранения каждый шестой человек в мире имеет ограничение здоровья чтобы интернет доступным для каждого консорциум всемирной паутины разработал минимальные требования руководство по обеспечению доступности веб-контенту 97% веб-страниц игнорируют эти требования такими сайтами не может пользоваться человек с ограничениями слуха зрения опорнодвигательного аппарата ментальными особенностями социальные сети вызывают зависимость научно доказано что яркие привлекательные интерфейсы мобильных приложений прово в нашем м

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Your job is to produce a final summary.
We have provided an existing summary up to a certain point: Write a concise summary of the following:


"соцсети небезопасны Травля угрозы преследования по данным социологических исследований более 10% пользователей интернета сталкивались с ситуациями представляющими реальную опасность и не менее 40% хотя бы раз страдали от нарушения личных границ в онлайн пространстве соцсети недоступны для всех по данным всемирной организации здравоохранения каждый шестой человек в мире имеет ограничение здоровья чтобы интернет доступным для каждого консорциум всемирной паутины разработал минимальные требования руководство по обеспечению доступности веб-контенту 97% веб-страниц игнорируют эти требования такими сайтами не может пользоваться человек с ограничениями слуха зрения опорнодвигательного аппарата ментальными особенностями социальные сети вызывают зависимость научно доказано ч

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Your job is to produce a final summary.
We have provided an existing summary up to a certain point: Your job is to produce a final summary.
We have provided an existing summary up to a certain point: Write a concise summary of the following:


"соцсети небезопасны Травля угрозы преследования по данным социологических исследований более 10% пользователей интернета сталкивались с ситуациями представляющими реальную опасность и не менее 40% хотя бы раз страдали от нарушения личных границ в онлайн пространстве соцсети недоступны для всех по данным всемирной организации здравоохранения каждый шестой человек в мире имеет ограничение здоровья чтобы интернет доступным для каждого консорциум всемирной паутины разработал минимальные требования руководство по обеспечению доступности веб-контенту 97% веб-страниц игнорируют эти требования такими сайтами не может пользоваться человек с ограничениями слуха зрения опорнодви

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Your job is to produce a final summary.
We have provided an existing summary up to a certain point: Your job is to produce a final summary.
We have provided an existing summary up to a certain point: Your job is to produce a final summary.
We have provided an existing summary up to a certain point: Write a concise summary of the following:


"соцсети небезопасны Травля угрозы преследования по данным социологических исследований более 10% пользователей интернета сталкивались с ситуациями представляющими реальную опасность и не менее 40% хотя бы раз страдали от нарушения личных границ в онлайн пространстве соцсети недоступны для всех по данным всемирной организации здравоохранения каждый шестой человек в мире имеет ограничение здоровья чтобы интернет доступным для каждого консорциум всемирной паутины разработал минимальные требования руководство по обеспечению доступности веб-контенту 97% веб-страниц игнорирую

In [53]:
chain.llm_chain.prompt.template = \
"""Создай краткое описание для следующего транскрипта:

"{text}"
"""

In [54]:
summary = chain.run(input_documents=texts)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Создай краткое описание для следующего транскрипта:

"соцсети небезопасны Травля угрозы преследования по данным социологических исследований более 10% пользователей интернета сталкивались с ситуациями представляющими реальную опасность и не менее 40% хотя бы раз страдали от нарушения личных границ в онлайн пространстве соцсети недоступны для всех по данным всемирной организации здравоохранения каждый шестой человек в мире имеет ограничение здоровья чтобы интернет доступным для каждого консорциум всемирной паутины разработал минимальные требования руководство по обеспечению доступности веб-контенту 97% веб-страниц игнорируют эти требования такими сайтами не может пользоваться человек с ограничениями слуха зрения опорнодвигательного аппарата ментальными особенностями социальные сети вызывают зависимость научно доказано что яркие привлекательные интерфейсы мобильных приложений прово в

In [56]:
summary

'Создай краткое описание для следующего транскрипта:\n\n"соцсети небезопасны Травля угрозы преследования по данным социологических исследований более 10% пользователей интернета сталкивались с ситуациями представляющими реальную опасность и не менее 40% хотя бы раз страдали от нарушения личных границ в онлайн пространстве соцсети недоступны для всех по данным всемирной организации здравоохранения каждый шестой человек в мире имеет ограничение здоровья чтобы интернет доступным для каждого консорциум всемирной паутины разработал минимальные требования руководство по обеспечению доступности веб-контенту 97% веб-страниц игнорируют эти требования такими сайтами не может пользоваться человек с ограничениями слуха зрения опорнодвигательного аппарата ментальными особенностями социальные сети вызывают зависимость научно доказано что яркие привлекательные интерфейсы мобильных приложений прово в нашем мозге выработку гормона дофамина гормона удовольствия результат настоящая зависимость наш органи